In [1]:
#token: 5a28fc93216d8b8a1c8345a2ddabd46b20fe3b02a13794c90c94ebe4e99ad9569fa84a271213a9393d6e9cb5f2900f62d61828d3ff50ce2dfff70d617463729e

In [2]:
import qiskit as q
%matplotlib inline

circuit = q.QuantumCircuit(2, 2) # 2 qubits, 2 classical bits

# apply not gate to qubit 0 to make its initial value of 0 a 1
# currently 0, 0
circuit.x(0)
# 1, 0

# cnot, controlled not. flips 2nd qubit value iff first qubit is a 1
# cnot can be used to entangle 
circuit.cx(0, 1)
# 1, 1

circuit.measure([0, 1], [0, 1])

circuit.draw()